# PAQUITA RELATION EXTRACTION

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## GRAPHICAL PROCESSING UNIT (GPU) & TENSOR PROCESSING UNIT (TPU)

In [2]:
#https://colab.research.google.com/notebooks/tpu.ipynb#scrollTo=hJl3vNtJOB-x

TPU_INICIADA = False #@param {type:"boolean"}


%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

if TPU_INICIADA:
  # try:
  #   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  #   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  # except ValueError:
  #   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

  # tf.config.experimental_connect_to_cluster(tpu)
  # tf.tpu.experimental.initialize_tpu_system(tpu)
  # tpu_strategy = tf.distribute.TPUStrategy(tpu)


  # Detect hardware, return appropriate distribution strategy
  try:
      tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # TPU detection
      print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  except ValueError:
      tpu = None

  if tpu:
      tf.config.experimental_connect_to_cluster(tpu)
      tf.tpu.experimental.initialize_tpu_system(tpu)
      tpu_strategy = strategy = tf.distribute.TPUStrategy(tpu)
  else:
      tpu_strategy = tf.distribute.get_strategy()

  print("REPLICAS: ", tpu_strategy.num_replicas_in_sync)


else:
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.12.0
Wed May 17 13:16:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+---------------

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## LET'S WORK (NAMED ENTITY RECOGNITION)
https://towardsdatascience.com/how-to-fine-tune-bert-transformer-with-spacy-3-6a90bfe57647

### IMPORTING LIBRARIES

In [4]:
!sudo mkdir -p /var/cuda-repo-11-8-local
!sudo wget -O /var/cuda-repo-11-8-local/3bf863cc.pub https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/3bf863cc.pub #7fa2af80.pub

--2023-05-17 13:16:48--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/3bf863cc.pub
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1650 (1.6K) [application/octet-stream]
Saving to: ‘/var/cuda-repo-11-8-local/3bf863cc.pub’

/var/cuda-repo-11-8 100%[===================>]   1.61K  --.-KB/s    in 0s      

2023-05-17 13:16:48 (156 MB/s) - ‘/var/cuda-repo-11-8-local/3bf863cc.pub’ saved [1650/1650]



In [5]:
!apt-key add /var/cuda-repo-11-8-local/3bf863cc.pub
!apt-get update
!apt-get install cuda-11.8

OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,690 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,167 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 

In [6]:
!pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118


In [7]:
!pip install -U spacy[cuda118,transformers]
!export CUDA_PATH="/usr/local/cuda-11.8"
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.7/190.7 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


In [8]:
!pip install cupy-cuda11x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### ADAPTING DATA TO SPACY

In [9]:
!python -m spacy convert /content/gdrive/MyDrive/Notebooks/Paquita/Spanish/NER_training.tsv ./ -t json -n 1 -c iob
!python -m spacy convert /content/gdrive/MyDrive/Notebooks/Paquita/Spanish/NER_TEST.tsv ./ -t json -n 1 -c iob
!python -m spacy convert /content/gdrive/MyDrive/Notebooks/Paquita/Spanish/NER_DEV.tsv ./ -t json -n 1 -c iob

2023-05-17 13:24:08.053849: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
⚠ No sentence boundaries found. Use `-s` to automatically segment
sentences.
✔ Generated output file (1 documents): NER_training.json
2023-05-17 13:24:24.738922: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
⚠ No sentence boundaries found. Use `-s` to automatically segment
sentences.
✔ Generated output file (1 documents): NER_TEST.json
2023-05-17 13:24:35.077626: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disab

In [10]:
!python -m spacy convert /content/NER_training.json ./ -t spacy
!python -m spacy convert /content/NER_TEST.json ./ -t spacy
!python -m spacy convert /content/NER_DEV.json ./ -t spacy


2023-05-17 13:24:45.528137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (118 documents): NER_training.spacy
2023-05-17 13:24:56.417831: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (98 documents): NER_TEST.spacy
2023-05-17 13:25:06.241458: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Generated output file (46 documents): NER_DEV.spacy


### TRAIN

In [11]:
import shutil
shutil.copy('/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/ner.cfg', '/content/ner.cfg')

'/content/ner.cfg'

In [12]:
!python -m spacy init fill-config /content/ner.cfg /content/ner.cfg

2023-05-17 13:25:16.701768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/ner.cfg
You can now add your data and train your pipeline:
python -m spacy train ner.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
# %env TOKENIZERS_PARALLELISM=false

In [14]:
!python -m spacy debug data /content/ner.cfg

2023-05-17 13:25:26.199085: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================ Data file validation ============================
Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
✔ Pipeline can be initialized with data
✔ Corpus is

In [15]:
import os
archivos = os.listdir()

if 'results' not in archivos:
  os.mkdir('results')
else:
  shutil.rmtree('results')


https://spacy.io/usage/training

In [16]:
!python -m spacy train /content/ner.cfg --output /content/results -g 0

# !python -m spacy train -g 0 /content/ner.cfg --paths.train  /content/NER_training.spacy --paths.dev /content/NER_DEV.spacy --output /content/results

2023-05-17 13:26:06.950222: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: /content/results
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-17 13:26:17,087] [INFO] Set up nlp object from config
[2023-05-17 13:26:17,103] [INFO] Pipeline: ['transformer', 'ner']
[2023-05-17 13:26:17,106] [INFO] Created vocabulary
[2023-05-17 13:26:17,107] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from 

In [17]:
import spacy
nlp = spacy.load("/content/results/model-best")

text = [
'''
- Paciente de 65 a. de edad, que presentaba una elevación progresiva de las cifras de PSA desde 6 ng/ml a 12 ng/ml en el último año. Dicho paciente había sido sometido un año antes a una biopsia transrectal de próstata ecodirigida por sextantes que fue negativa.  Se decide, ante la elevación del PSA, realizar una E-RME previa a la 2ª biopsia transrectal, en la que se objetiva una lesión hipointensa que abarca zona central i periférica del ápex del lóbulo D prostático. El estudio espectroscópico de ésta lesión mostró una curva de colina discretamente más elevada que la curva de citrato, con un índice de Ch-Cr/Ci > 0,80, que sugería la presencia de lesión neoplásica, por lo que se biopsia dicha zona por ecografía transrectal. La AP de la biopsia confirmó la presencia de un ADK próstata Gleason 6.
- Paciente de 19 años de edad, sin antecedentes de interés, que consulta por tumoración ligeramente dolorosa en el testículo derecho, de unos 8 meses de evolución, encontrándose en la exploración clínica un aumento testicular con zonas nodulares. El testículo izquierdo no muestra alteraciones. Alfa-Feto-Proteina: 38,2 y Beta-HGC de 1,48. Ecográficamente se observan tres nódulos tumorales independientes y bien delimitados: dos de ellos heterogéneos, sólidos, de 20 y 33 mm de diámetros, con áreas quísticas y calcificaciones. El tercer nódulo era sólido hipoecoico y homogéneo, de 26 mm de diámetro. Todos los nódulos presentaban un aumento de la vascularización con flujos arteriales de baja resistencia. El estudio de extensión es negativo. Se realiza orquiectomía. Histológicamente los dos primeros nódulos correspondían a teratocarcinomas (teratoma maduro y carcinoma embrionario predominante), siendo macroscópicamente masas pardo-blanquecinas, con cavidades de contenido seroso amarillento, calcificaciones y focos de necrosis. El tercero fué diagnosticado como seminoma clásico, siendo más homogéneo macroscópicamente.
- Mujer de 58 años de edad que es estudiada por el Servicio de digestivo por dolor en hipocondrio derecho de 6 meses de evolución, acompañándose de alteración del hábito intestinal. No presenta síndrome general ni clínica urológica.  A la exploración física impresiona de buen estado general, normohidratada y normocoloreada y con obesidad moderada. No existe hábito cushingoide ni estigmas de virilización. Las cifras tensionales son normales.  La auscultación cardiopulmonar es normal. El abdomen es globuloso, blando y depresible, dificultando la palpación de posibles masas u organomegalias, mostrando leve dolor a la presión profunda en hipocondrio derecho, no presentando signo de Murphy.  En cuanto a las exploraciones complementarias, el análisis rutinario bioquímico y hematológico, así como las pruebas de función suprarrenal se encuentran en rangos de normalidad.  La ecografía demuestra la presencia de una tumoración en el área suprarrenal derecha de aproximadamente 6x6 cm. No se observan metástasis hepáticas.  La TAC confirma la masa referida, en íntimo contacto con la pared de vena cava inferior a la que aparentemente desplaza, sin poder descartarse infiltración de la misma. No existe evidencia de afectación ganglionar retroperitoneal ni metástasis a otro nivel. No se demuestra asimismo presencia de trombosis tumoral.  Ante estos hallazgos se decide practicar RMN para una correcta definición de la afectación de tejidos vecinos, informando de idénticos hallazgos de la TAC y sin poder predecir de modo definitivo la infiltración parietal.  Con el diagnóstico de masa suprarrenal se decide intervención, que se lleva a cabo a través de una incisión subcostal derecha transperitoneal. Se aborda la región suprarrenal derecha, evidenciándose una voluminosa tumoración a dicho nivel. Se inicia la disección de la misma, no hallándose plano de separación en la zona de contacto con la pared lateral y posterior de vena cava, por lo que se decide exéresis de la misma tras control venoso proximal y distal a la masa y con sutura con polipropileno de la pared de la vena cava inferior.  La paciente sigue un curso postoperatorio favorable, sin desarrollar complicaciones. Cursa alta al séptimo día postintervención.  El estudio histológico macroscópico demuestra una pieza de 6 x 8 cm. de consistencia elástica y fácil corte, de aspecto carnoso y coloración pardo-grisácea. Se reconocen restos de tejido suprarrenal en periferia, de aspecto aparentemente normal.  La microscopía nos informa de una proliferación de elementos celulares de estirpe estromal, con núcleos pleomórficos de gran tamaño, con células multinucleadas que adoptan un patrón fascicular. El índice mitótico es elevado ( 7 m/ 10 campos de gran aumento). Existen zonas de necrosis intratumoral e invasión de tejido suprarrenal presente en área periférica.  El diagnóstico definitivo es de LEIOMIOSARCOMA DE CAVA.  En cuanto a la evolución, los controles, tanto analíticos como de imagen, realizados en el período postquirúrgico precoz muestran normalidad. Se remite el caso al Servicio de Oncología que decide tratamiento radioterápico de consolidación sobre lecho quirúrgico, iniciándose éste a los 40 días tras la intervención.  Se realizaron nuevos controles a los seis, nueve, doce y veinticuatro meses, resultando todos ellos normales.  Actualmente la paciente se encuentra viva y libre de enfermedad, tras 28 meses de seguimiento.
'''
]


for doc in nlp.pipe(text):
       print(f"spans: {[(e.start,e.end,  e.text, e.label_) for e in doc.ents]}")


spans: [(16, 17, 'cifras', 'EVENT'), (18, 19, 'PSA', 'EVENT'), (20, 24, '6 ng/ml', 'RML'), (25, 29, '12 ng/ml', 'RML'), (53, 54, 'negativa', 'RML'), (97, 98, 'estudio', 'EVENT'), (118, 119, 'índice', 'EVENT'), (123, 126, '> 0,80,', 'RML'), (155, 156, 'ADK', 'EVENT'), (157, 158, 'Gleason', 'EVENT'), (158, 161, '6.\n', 'RML'), (186, 191, 'unos 8 meses de evolución', 'RML'), (211, 212, 'Alfa-Feto-Proteina', 'EVENT'), (213, 214, '38,2', 'RML'), (215, 216, 'Beta-HGC', 'EVENT'), (217, 220, '1,48. Ecográficamente', 'RML'), (223, 224, 'nódulos', 'EVENT'), (235, 236, 'sólidos', 'EVENT'), (238, 244, '20 y 33 mm de diámetros', 'RML'), (255, 256, 'sólido', 'EVENT'), (261, 265, '26 mm de diámetro', 'RML'), (276, 277, 'flujos', 'EVENT'), (283, 284, 'estudio', 'EVENT'), (287, 288, 'negativo', 'RML'), (312, 313, 'masas', 'EVENT'), (412, 413, 'cifras', 'EVENT'), (415, 416, 'normales', 'RML'), (422, 423, 'normal', 'RML'), (477, 478, 'pruebas', 'EVENT'), (503, 506, 'aproximadamente 6x6 cm', 'RML'), (518,

In [18]:
from distutils.dir_util import copy_tree

origen = '/content/results'
destino = '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner'

copy_tree(origen, destino)

['/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/vocab/strings.json',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/vocab/key2row',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/vocab/vectors',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/vocab/vectors.cfg',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/vocab/lookups.bin',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/tokenizer',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/meta.json',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/config.cfg',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/ner/model',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-last/ner/cfg',
 '/content/gdrive/

## LET'S WORK (RELATION EXTRACTION)
https://github.com/explosion/projects/tree/v3/tutorials/rel_component

https://towardsdatascience.com/how-to-train-a-joint-entities-and-relation-extraction-classifier-using-bert-transformer-with-spacy-49eb08d91b5c

### IMPORT LIBRARIES

In [19]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [20]:
# !pip install -U spacy-nightly==3.0.0rc3 --pre
# !pip install spacy-nightly

In [21]:
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-17 13:33:58.018137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Cloned 'tutorials/rel_component' from 'explosion/projects' (branch
'v3')
/content/rel_component
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/rel_component


In [22]:
# !python -m spacy download es_dep_news_trf # Cambiar
!pip install -U spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### TRAIN

In [23]:
import shutil
shutil.copy('/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/rel_trf.cfg', '/content/rel_component/configs/rel_trf.cfg')

'/content/rel_component/configs/rel_trf.cfg'

In [24]:
%cd rel_component

/content/rel_component


In [25]:
import os
import shutil 

archivos = os.listdir()

if 'data' not in archivos:
  os.mkdir('data')
else:
  shutil.rmtree('data')

In [26]:
data_path = '/content/rel_component/data'


def is_directory_empty(directory):
    return not os.listdir(directory)




if is_directory_empty(data_path):
  shutil.copy('/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/relations_dev.spacy', data_path) #+'/relations_dev.spacy'
  shutil.copy('/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/relations_test.spacy', data_path) #+'/relations_test.spacy'
  shutil.copy('/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/relations_training.spacy', data_path) #+'/relations_training.spacy'
else:
    print('El directorio no está vacío')
  

Modify paths on `proyect.yml`:

train_file: "data/relations_training.spacy"

dev_file: "data/relations_dev.spacy"

test_file: "data/relations_test.spacy"

In [27]:
%env LC_ALL=C.UTF8
%env LANG=C.UTF8

env: LC_ALL=C.UTF8
env: LANG=C.UTF8


In [28]:
!locale -a

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (C.UTF8)
locale: Cannot set LC_CTYPE to default locale: No such file or directory
locale: Cannot set LC_MESSAGES to default locale: No such file or directory
locale: Cannot set LC_COLLATE to default locale: No such file or directory
C
C.UTF-8
POSIX
en_US.utf8


In [29]:
!spacy project run train_gpu # command to train train transformers

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (C.UTF8)
2023-05-17 13:34:50.515112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

================================= train_gpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_trf.cfg --output training --paths.train data/relations_training.spacy --paths.dev data/relations_dev.spacy -c ./scripts/custom_functions.py --gpu-id 0
2023-05-17 13:34:59.010481: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-17 13:35:05,744] [INFO] Set up nlp object from config
[2023-05-17 13:35:05,761] [INFO] Pipeline: ['transformer', 'relation_extractor']
[2023-05-17 13:35:05,765] [INFO] Created vocabulary
[2023-05-17 13:35:05,765] [INFO] Finished initializing nlp obje

In [30]:
!spacy project run evaluate # command to evaluate on test dataset

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (C.UTF8)
2023-05-17 13:44:26.130041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

================================== evaluate ==================================
Running command: /usr/bin/python3 ./scripts/evaluate.py training/model-best data/relations_test.spacy False
2023-05-17 13:44:34.645824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Random baseline:
threshold 0.00 	 {'rel_micro_p': '8.23', 'rel_micro_r': '100.00', 'rel_micro_f': '15.21'}
threshold 0.05 	 {'rel_micro_p': '8.37', 'rel_micro_r': '96.97', 'rel_micro_f': '15.41'}
threshold 0.10 	 {'rel_micro_p': '8.48', 'rel_micro_r': '92.73', 'rel_micro_f': '15.54'}
threshold 0.20 	 {'rel_micro_p': '8.41', 'rel_micro_r': '82.42', 'rel_micro_f': '15.26'}
threshold 0.30 	 {'rel_micro_p': '8.08', 'rel_micro_r': '69.70', 'rel_micro_f': '14.47'}
threshold 0.40 	 {'rel_micro_p'

In [31]:
from distutils.dir_util import copy_tree

origen = '/content/rel_component/training'
destino = '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re'

copy_tree(origen, destino)

['/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/vocab/strings.json',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/vocab/key2row',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/vocab/vectors',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/vocab/vectors.cfg',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/vocab/lookups.bin',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/relation_extractor/model',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/relation_extractor/cfg',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/tokenizer',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/meta.json',
 '/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-last/config.cfg'

# PRODUCTION WORKFLOW

## NER

In [32]:
import spacy
nlp = spacy.load("/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_ner/model-best")

text = [
'''
- Paciente de 65 a. de edad, que presentaba una elevación progresiva de las cifras de PSA desde 6 ng/ml a 12 ng/ml en el último año. Dicho paciente había sido sometido un año antes a una biopsia transrectal de próstata ecodirigida por sextantes que fue negativa.  Se decide, ante la elevación del PSA, realizar una E-RME previa a la 2ª biopsia transrectal, en la que se objetiva una lesión hipointensa que abarca zona central i periférica del ápex del lóbulo D prostático. El estudio espectroscópico de ésta lesión mostró una curva de colina discretamente más elevada que la curva de citrato, con un índice de Ch-Cr/Ci > 0,80, que sugería la presencia de lesión neoplásica, por lo que se biopsia dicha zona por ecografía transrectal. La AP de la biopsia confirmó la presencia de un ADK próstata Gleason 6.
- Paciente de 19 años de edad, sin antecedentes de interés, que consulta por tumoración ligeramente dolorosa en el testículo derecho, de unos 8 meses de evolución, encontrándose en la exploración clínica un aumento testicular con zonas nodulares. El testículo izquierdo no muestra alteraciones. Alfa-Feto-Proteina: 38,2 y Beta-HGC de 1,48. Ecográficamente se observan tres nódulos tumorales independientes y bien delimitados: dos de ellos heterogéneos, sólidos, de 20 y 33 mm de diámetros, con áreas quísticas y calcificaciones. El tercer nódulo era sólido hipoecoico y homogéneo, de 26 mm de diámetro. Todos los nódulos presentaban un aumento de la vascularización con flujos arteriales de baja resistencia. El estudio de extensión es negativo. Se realiza orquiectomía. Histológicamente los dos primeros nódulos correspondían a teratocarcinomas (teratoma maduro y carcinoma embrionario predominante), siendo macroscópicamente masas pardo-blanquecinas, con cavidades de contenido seroso amarillento, calcificaciones y focos de necrosis. El tercero fué diagnosticado como seminoma clásico, siendo más homogéneo macroscópicamente.
- Mujer de 58 años de edad que es estudiada por el Servicio de digestivo por dolor en hipocondrio derecho de 6 meses de evolución, acompañándose de alteración del hábito intestinal. No presenta síndrome general ni clínica urológica.  A la exploración física impresiona de buen estado general, normohidratada y normocoloreada y con obesidad moderada. No existe hábito cushingoide ni estigmas de virilización. Las cifras tensionales son normales.  La auscultación cardiopulmonar es normal. El abdomen es globuloso, blando y depresible, dificultando la palpación de posibles masas u organomegalias, mostrando leve dolor a la presión profunda en hipocondrio derecho, no presentando signo de Murphy.  En cuanto a las exploraciones complementarias, el análisis rutinario bioquímico y hematológico, así como las pruebas de función suprarrenal se encuentran en rangos de normalidad.  La ecografía demuestra la presencia de una tumoración en el área suprarrenal derecha de aproximadamente 6x6 cm. No se observan metástasis hepáticas.  La TAC confirma la masa referida, en íntimo contacto con la pared de vena cava inferior a la que aparentemente desplaza, sin poder descartarse infiltración de la misma. No existe evidencia de afectación ganglionar retroperitoneal ni metástasis a otro nivel. No se demuestra asimismo presencia de trombosis tumoral.  Ante estos hallazgos se decide practicar RMN para una correcta definición de la afectación de tejidos vecinos, informando de idénticos hallazgos de la TAC y sin poder predecir de modo definitivo la infiltración parietal.  Con el diagnóstico de masa suprarrenal se decide intervención, que se lleva a cabo a través de una incisión subcostal derecha transperitoneal. Se aborda la región suprarrenal derecha, evidenciándose una voluminosa tumoración a dicho nivel. Se inicia la disección de la misma, no hallándose plano de separación en la zona de contacto con la pared lateral y posterior de vena cava, por lo que se decide exéresis de la misma tras control venoso proximal y distal a la masa y con sutura con polipropileno de la pared de la vena cava inferior.  La paciente sigue un curso postoperatorio favorable, sin desarrollar complicaciones. Cursa alta al séptimo día postintervención.  El estudio histológico macroscópico demuestra una pieza de 6 x 8 cm. de consistencia elástica y fácil corte, de aspecto carnoso y coloración pardo-grisácea. Se reconocen restos de tejido suprarrenal en periferia, de aspecto aparentemente normal.  La microscopía nos informa de una proliferación de elementos celulares de estirpe estromal, con núcleos pleomórficos de gran tamaño, con células multinucleadas que adoptan un patrón fascicular. El índice mitótico es elevado ( 7 m/ 10 campos de gran aumento). Existen zonas de necrosis intratumoral e invasión de tejido suprarrenal presente en área periférica.  El diagnóstico definitivo es de LEIOMIOSARCOMA DE CAVA.  En cuanto a la evolución, los controles, tanto analíticos como de imagen, realizados en el período postquirúrgico precoz muestran normalidad. Se remite el caso al Servicio de Oncología que decide tratamiento radioterápico de consolidación sobre lecho quirúrgico, iniciándose éste a los 40 días tras la intervención.  Se realizaron nuevos controles a los seis, nueve, doce y veinticuatro meses, resultando todos ellos normales.  Actualmente la paciente se encuentra viva y libre de enfermedad, tras 28 meses de seguimiento.
'''
]


for doc in nlp.pipe(text):
    print([(ent.text, ent.label_) for ent in doc.ents])

[('cifras', 'EVENT'), ('PSA', 'EVENT'), ('6 ng/ml', 'RML'), ('12 ng/ml', 'RML'), ('negativa', 'RML'), ('estudio', 'EVENT'), ('índice', 'EVENT'), ('> 0,80,', 'RML'), ('ADK', 'EVENT'), ('Gleason', 'EVENT'), ('6.\n', 'RML'), ('unos 8 meses de evolución', 'RML'), ('Alfa-Feto-Proteina', 'EVENT'), ('38,2', 'RML'), ('Beta-HGC', 'EVENT'), ('1,48. Ecográficamente', 'RML'), ('nódulos', 'EVENT'), ('sólidos', 'EVENT'), ('20 y 33 mm de diámetros', 'RML'), ('sólido', 'EVENT'), ('26 mm de diámetro', 'RML'), ('flujos', 'EVENT'), ('estudio', 'EVENT'), ('negativo', 'RML'), ('masas', 'EVENT'), ('cifras', 'EVENT'), ('normales', 'RML'), ('normal', 'RML'), ('pruebas', 'EVENT'), ('aproximadamente 6x6 cm', 'RML'), ('masa', 'EVENT'), ('polipropileno', 'EVENT'), ('6 x 8 cm', 'RML'), ('normal', 'RML'), ('núcleos', 'EVENT'), ('células', 'EVENT'), ('índice', 'EVENT'), ('7 m/ 10 campos de gran aumento)', 'RML'), ('controles', 'EVENT'), ('normales', 'RML')]


### REL

In [33]:
%cd /content

/content


In [34]:
# shutil.copy('/content/rel_component/scripts/rel_model.py', '/content/rel_component/rel_model.py') 
# shutil.copy('/content/rel_component/scripts/rel_pipe.py', '/content/rel_component/rel_pipe.py') 

shutil.copy('/content/rel_component/scripts/rel_model.py', '/content/rel_model.py') 
shutil.copy('/content/rel_component/scripts/rel_pipe.py', '/content/rel_pipe.py') 

'/content/rel_pipe.py'

In [35]:
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors


# We load the relation extraction (REL) model
nlp2 = spacy.load("/content/gdrive/MyDrive/Notebooks/Paquita/Spanish/results_spanish_re/model-best")
nlp2.add_pipe('sentencizer')

# We take the entities generated from the NER pipeline and input them to the REL pipeline
for name, proc in nlp2.pipeline:
  doc = proc(doc) 

# Here, we split the paragraph into sentences and apply the relation extraction for each pair of entities found in each sentence.
for value, rel_dict in doc._.rel.items():
  for sent in doc.sents:
    for e in sent.ents:
      for b in sent.ents:
        if e.start == value[0] and b.start == value[1]:
          if rel_dict['REL'] >=0.1: # Modify the threshold to take every relation in text
            print(f" entities: {e.text, b.text} --> predicted relation: {rel_dict}")

 entities: ('cifras', 'PSA') --> predicted relation: {'REL': 0.28692457}
 entities: ('cifras', '6 ng/ml') --> predicted relation: {'REL': 0.8611266}
 entities: ('cifras', '12 ng/ml') --> predicted relation: {'REL': 0.88203216}
 entities: ('estudio', 'índice') --> predicted relation: {'REL': 0.41887906}
 entities: ('estudio', '> 0,80,') --> predicted relation: {'REL': 0.9948822}
 entities: ('índice', '> 0,80,') --> predicted relation: {'REL': 0.95678353}
 entities: ('Alfa-Feto-Proteina', '38,2') --> predicted relation: {'REL': 0.25904912}
 entities: ('Beta-HGC', '38,2') --> predicted relation: {'REL': 0.35762006}
 entities: ('nódulos', 'sólidos') --> predicted relation: {'REL': 0.609802}
 entities: ('nódulos', '20 y 33 mm de diámetros') --> predicted relation: {'REL': 0.9874013}
 entities: ('sólidos', '20 y 33 mm de diámetros') --> predicted relation: {'REL': 0.115221374}
 entities: ('sólido', '26 mm de diámetro') --> predicted relation: {'REL': 0.47750515}
 entities: ('estudio', 'negat